In [1]:
from bs4 import BeautifulSoup

html = '''
<table class="profile-table profile-table_colors">
    <thead>
    <tr>
        <th class="profile-table_colors__th_1">Period</th>
        <th class="profile-table_colors__th_2">Standard</th>
        <th class="profile-table_colors__th_3">Rapid</th>
        <th class="profile-table_colors__th_4">Blitz</th>
    </tr>
    </thead>
    <tbody>
        <tr><td align="right">October 2024 (tentative)</td><td>No Games</td><td>No Games</td><td>No Games</td></tr>
        <tr><td align="right">September 2024</td><td>No Games</td><td>Available 7.1</td><td>Available 0.3</td></tr>
        <tr><td align="right">August 2024</td><td>No Games</td><td>No Games</td><td>No Games</td></tr>
        <!-- Other rows -->
    </tbody>
</table>
'''

# پارس کردن HTML
soup = BeautifulSoup(html, 'html.parser')

# پیدا کردن جدول
table = soup.find('table', {'class': 'profile-table'})

# استخراج داده‌ها
headers = [th.text for th in table.find('thead').find_all('th')]
rows = []
for row in table.find('tbody').find_all('tr'):
    rows.append([td.text for td in row.find_all('td')])

# چاپ جدول
print(headers)
for row in rows:
    print(row)


['Period', 'Standard', 'Rapid', 'Blitz']
['October 2024 (tentative)', 'No Games', 'No Games', 'No Games']
['September 2024', 'No Games', 'Available 7.1', 'Available 0.3']
['August 2024', 'No Games', 'No Games', 'No Games']


In [2]:
import pandas as pd

df=pd.read_html("p1.html")
len(df)

C:\Users\PC\AppData\Local\Temp\ipykernel_5060\866243129.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df=pd.read_html("p1.html")


ImportError: Missing optional dependency 'lxml'.  Use pip or conda to install lxml.

In [ ]:
for i,dfx in enumerate(df):
    print(i)
    display(dfx)

In [ ]:
df[5]

In [ ]:
df[5].to_excel("dfon.xlsx")


In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# خواندن فایل و استخراج ID‌ها
def extract_ids_from_txt(file_path):
    ids = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines[1:]:  # خط اول را نادیده می‌گیریم چون عنوان‌هاست
            id_part = line.split()[0]  # اولین ستون که ID است
            ids.append(id_part)
    return ids

# تابع برای دریافت HTML از URL و استخراج جداول با پانداس
def get_tables_from_url(player_id):
    url = f"https://ratings.fide.com/profile/{player_id}/calculations"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # استخراج جداول از HTML
    tables = pd.read_html(response.text)
    
    # نمایش جدولی که ستون 'Period' داشته باشد
    for i, table in enumerate(tables):
        if 'Period' in table.columns:  # چک کردن وجود ستون 'Period'
            print(f"Table {i} for Player ID {player_id}:\n")
            display(table)
            break  # اگر فقط اولین جدول با این شرط را می‌خواهید نشان دهید

# خواندن لیست ID‌ها از فایل
file_path = 'sr.txt'
player_ids = extract_ids_from_txt(file_path)

# پیمایش لیست و دریافت اطلاعات هر بازیکن
con=0
for player_id in player_ids:
    print(f"Fetching data for Player ID: {player_id}")
    get_tables_from_url(player_id)
    if con>10:
        con+=1
        break  # اگر فقط برای اولین بازیکن می‌خواهید انجام دهید


# MAIN

In [2]:
import aiohttp
import asyncio
import pandas as pd
from bs4 import BeautifulSoup
import os


# تابع برای خواندن فایل و استخراج ID‌ها
def extract_ids_from_txt(file_path):
    ids = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines[1:]:  # خط اول را نادیده می‌گیریم چون عنوان‌هاست
            id_part = line.split()[0]  # اولین ستون که ID است
            ids.append(id_part)
    return ids

# تابع async برای دریافت HTML از URL و استخراج جدول با ستون 'Period'
async def fetch_table_with_period(session, player_id, semaphore):
    print(player_id)
    url = f"https://ratings.fide.com/profile/{player_id}/calculations"
    async with semaphore:  # محدود کردن تعداد درخواست‌ها به 1000 همزمان
        async with session.get(url) as response:
            print(url)
            if response.status == 200:
                html = await response.text()
                tables = pd.read_html(html)
                for table in tables:
                    if 'Period' in table.columns:  # چک کردن وجود ستون 'Period'
                        print(f"Player ID {player_id} has 'Period' table:\n")
                        display(table.head(10))
                        table.to_excel(f"A:/payan_name/python-scrapping/excel/{player_id}.xlsx")
                        return table
            else:
                print(f"Failed to fetch data for Player ID: {player_id} - Status code: {response.status}")

# تابع async برای مدیریت دانلودها
async def fetch_all_tables_with_period(player_ids, max_concurrent_tasks=1000):
    semaphore = asyncio.Semaphore(max_concurrent_tasks)  # محدودیت 1000 درخواست همزمان
    async with aiohttp.ClientSession() as session:
        tasks = []
        for player_id in player_ids:
            task = fetch_table_with_period(session, player_id, semaphore)
            tasks.append(task)
        await asyncio.gather(*tasks)  # منتظر تمام شدن همه وظایف async


# تابع برای دریافت نام فایل‌های موجود در فولدر 'excel'
def get_existing_files_in_excel_folder(folder_path):
    files = os.listdir(folder_path)  # گرفتن تمام فایل‌های موجود در فولدر
    excel_files = [f.replace('.xlsx', '') for f in files if f.endswith('.xlsx')]  # حذف پسوند '.xlsx' از نام فایل‌ها
    return excel_files


# تابع اصلی برای اجرا (همراه با بررسی حلقه رویداد موجود)
def main(file_path):
        # استخراج ID‌ها از فایل
    player_ids = extract_ids_from_txt(file_path)
    print(f"Initial Player IDs: {len(player_ids)}")

    # خواندن فایل‌های موجود در پوشه excel و حذف آن‌ها از player_ids
    existing_files = get_existing_files_in_excel_folder('excel')
    print(f"Existing files in excel folder: {len(existing_files)}")

    # حذف بازیکنان که فایل مربوطه‌شان در فولدر موجود است
    player_ids = [player_id for player_id in player_ids if player_id not in existing_files]
    print(f"Remaining Player IDs after removing existing files: {len(player_ids)}")

    print(len(player_ids))
    player_ids=player_ids[:10]
    try:
        # بررسی اینکه آیا حلقه رویداد در حال اجراست یا خیر
        loop = asyncio.get_running_loop()
    except RuntimeError:  # اگر حلقه‌ای وجود نداشت، یکی ایجاد می‌کنیم
        loop = None
    
    if loop and loop.is_running():
        print("Using existing event loop.")
        # در صورتی که حلقه‌ای در حال اجرا باشد، از create_task برای اجرای وظیفه استفاده می‌کنیم
        task = loop.create_task(fetch_all_tables_with_period(player_ids))
    else:
        print("Starting new event loop.")
        # اگر حلقه‌ای در حال اجرا نبود، از asyncio.run استفاده می‌کنیم
        asyncio.run(fetch_all_tables_with_period(player_ids))

# اجرا
file_path = 'player.txt'
main(file_path)


Initial Player IDs: 417311
Existing files in excel folder: 1351
Remaining Player IDs after removing existing files: 415960
415960
Using existing event loop.


9211578
10646299
10663720
8108803
12211923
12215635
13712098
13836633
42560136
5302129
https://ratings.fide.com/profile/12211923/calculations
Player ID 12211923 has 'Period' table:



C:\Users\PC\AppData\Local\Temp\ipykernel_14700\2952414383.py:27: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)


,Period,RTNG,GMS,RAPID RTNG,RAPID GMS,BLITZ RTNG,BLITZ GMS
0,2024-Sep,1660,0,NaN,NaN,NaN,NaN
1,2024-Aug,1660,0,NaN,NaN,NaN,NaN
2,2024-Jul,1660,0,NaN,NaN,NaN,NaN
3,2024-Jun,1660,0,NaN,NaN,NaN,NaN
4,2024-May,1660,0,NaN,NaN,NaN,NaN
5,2024-Apr,1660,0,NaN,NaN,NaN,NaN
6,2024-Mar,1660,0,NaN,NaN,NaN,NaN
7,2024-Feb,1434,0,NaN,NaN,NaN,NaN
8,2024-Jan,1434,0,NaN,NaN,NaN,NaN
9,2023-Dec,1434,0,NaN,NaN,NaN,NaN


https://ratings.fide.com/profile/13712098/calculations
https://ratings.fide.com/profile/10663720/calculations
Player ID 10663720 has 'Period' table:



...

Player ID 13712098 has 'Period' table:



,Period,RTNG,GMS,RAPID RTNG,RAPID GMS,BLITZ RTNG,BLITZ GMS
0,2024-Sep,1699.0,0.0,1612,0,1637.0,0.0
1,2024-Aug,1699.0,0.0,1612,0,1637.0,7.0
2,2024-Jul,1699.0,0.0,1612,0,1610.0,0.0
3,2024-Jun,1699.0,0.0,1612,0,1610.0,0.0
4,2024-May,1699.0,0.0,1612,0,1610.0,0.0
5,2024-Apr,1699.0,0.0,1612,4,1610.0,7.0
6,2024-Mar,1699.0,0.0,1639,0,1636.0,0.0
7,2024-Feb,1498.0,0.0,1399,0,1394.0,0.0
8,2024-Jan,1498.0,0.0,1399,0,1394.0,0.0
9,2023-Dec,1498.0,0.0,1399,0,1394.0,0.0


C:\Users\PC\AppData\Local\Temp\ipykernel_14700\2952414383.py:27: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html)


Player ID 5302129 has 'Period' table:



,Period,RTNG,GMS,RAPID RTNG,RAPID GMS,BLITZ RTNG,BLITZ GMS
0,2024-Sep,1835,0,1780,0,1791,0
1,2024-Aug,1835,0,1780,0,1791,0
2,2024-Jul,1835,0,1780,0,1791,0
3,2024-Jun,1835,0,1780,0,1791,0
4,2024-May,1835,0,1780,0,1791,0
5,2024-Apr,1835,0,1780,0,1791,0
6,2024-Mar,1835,0,1780,0,1791,0
7,2024-Feb,1725,0,1633,0,1651,0
8,2024-Jan,1725,0,1633,9,1651,0
9,2023-Dec,1725,0,1703,0,1651,5


In [ ]:
    # نمایش نتایج نهایی
    for player_id, table in results:
        if table is not None:
            print(f"Player ID: {player_id}\nTable:\n{table.head()}\n")
        else:
            print(f"No table found for Player ID: {player_id}\n")